#Obteniendo Datos de Phishing Mundiales con la API de PhishStats
[Articulo de Medium](https://)

Autor: Carlos Velázquez V

In [ ]:
#Realizando los imports necesarios 
import requests
import pandas as pd

In [ ]:
#Realizando solicitud 
response = requests.get("https://phishstats.info:2096/api/phishing")
print(response.json())

[{'id': 2, 'url': 'http://00000-00.000webhostapp.com/', 'ip': '2a02:4780:dead:6c62::1', 'countrycode': 'US', 'countryname': 'United States', 'regioncode': '', 'regionname': '', 'city': '', 'zipcode': '', 'latitude': '38', 'longitude': '-97.82200000', 'asn': 'AS47583', 'bgp': '2a02:4780:dead::/48', 'isp': 'HOSTINGER-AS, LT', 'title': None, 'date': '2017-05-26T03:00:00.000Z', 'date_update': '2020-12-08T01:50:24.000Z', 'hash': '9b4aa54561d60ebbf98677565c30330621c40bd10dc74da9e387a85809ec5ecc', 'score': None, 'host': '00000-00.000webhostapp.com', 'domain': None, 'tld': 'com', 'domain_registered_n_days_ago': None, 'screenshot': None, 'abuse_contact': None, 'ssl_issuer': None, 'ssl_subject': None, 'alexa_rank_host': None, 'alexa_rank_domain': None, 'n_times_seen_ip': None, 'n_times_seen_host': None, 'n_times_seen_domain': None, 'http_code': None, 'http_server': None, 'google_safebrowsing': None, 'virus_total': None, 'abuse_ch_malware': None, 'threat_crowd': None, 'threat_crowd_subdomain_coun

In [ ]:
#Convirtiendo a Pandas DataFrame 
df_response = pd.DataFrame.from_records(response.json())
df_response.head()

,id,url,ip,countrycode,countryname,regioncode,regionname,city,zipcode,latitude,longitude,asn,bgp,isp,title,date,date_update,hash,score,host,domain,tld,domain_registered_n_days_ago,screenshot,abuse_contact,ssl_issuer,ssl_subject,alexa_rank_host,alexa_rank_domain,n_times_seen_ip,n_times_seen_host,n_times_seen_domain,http_code,http_server,google_safebrowsing,virus_total,abuse_ch_malware,threat_crowd,threat_crowd_subdomain_count,threat_crowd_votes,vulns,ports,os,tags,technology,page_text
0,2,http://00000-00.000webhostapp.com/,2a02:4780:dead:6c62::1,US,United States,,,,,38,-97.82200000,AS47583,2a02:4780:dead::/48,"HOSTINGER-AS, LT",None,2017-05-26T03:00:00.000Z,2020-12-08T01:50:24.000Z,9b4aa54561d60ebbf98677565c30330621c40bd10dc74d...,None,00000-00.000webhostapp.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
1,3,http://000gbf1.wcomhost.com/3K8D4N2/index.php,208.91.197.27,US,United States,,,,,38,-97.82200000,AS40034,208.91.196.0/23,CONFLUENCE-NETWORK-INC - Confluence Networks I...,None,2017-02-04T02:00:00.000Z,2020-12-08T01:50:24.000Z,2d0796d76d0bc902c2df3553198df2ed4e86f8117101f0...,None,000gbf1.wcomhost.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
2,4,http://000gbf1.wcomhost.com/3K8D4N2/signin/222...,208.91.197.27,US,United States,,,,,38,-97.82200000,AS40034,208.91.196.0/23,CONFLUENCE-NETWORK-INC - Confluence Networks I...,None,2017-02-04T02:00:00.000Z,2020-12-08T01:50:24.000Z,e3bbc26bb59186f749d50b32636187d5152ee6ca6d9709...,None,000gbf1.wcomhost.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
3,5,http://000gbf1.wcomhost.com/3K8D4N2/signin/259...,208.91.197.27,US,United States,,,,,38,-97.82200000,AS40034,208.91.196.0/23,CONFLUENCE-NETWORK-INC - Confluence Networks I...,None,2017-02-04T02:00:00.000Z,2020-12-08T01:50:24.000Z,ba66742ea0fd1fd63c7f88f2debeb566364346266f26d0...,None,000gbf1.wcomhost.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
4,6,http://000hejn.wcomhost.com/PL/signin/DAD782MD...,208.91.197.27,US,United States,,,,,38,-97.82200000,AS40034,208.91.196.0/23,CONFLUENCE-NETWORK-INC - Confluence Networks I...,None,2017-05-01T03:00:00.000Z,2020-12-08T01:50:24.000Z,22b1c4a94c1279287a180d276398070ca7f6710b77fbf1...,None,000hejn.wcomhost.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,


In [ ]:
#Viendo los distintos nombres de las columnas
print(df_response.columns)

Index(['id', 'url', 'ip', 'countrycode', 'countryname', 'regioncode',
       'regionname', 'city', 'zipcode', 'latitude', 'longitude', 'asn', 'bgp',
       'isp', 'title', 'date', 'date_update', 'hash', 'score', 'host',
       'domain', 'tld', 'domain_registered_n_days_ago', 'screenshot',
       'abuse_contact', 'ssl_issuer', 'ssl_subject', 'alexa_rank_host',
       'alexa_rank_domain', 'n_times_seen_ip', 'n_times_seen_host',
       'n_times_seen_domain', 'http_code', 'http_server',
       'google_safebrowsing', 'virus_total', 'abuse_ch_malware',
       'threat_crowd', 'threat_crowd_subdomain_count', 'threat_crowd_votes',
       'vulns', 'ports', 'os', 'tags', 'technology', 'page_text'],
      dtype='object')


In [ ]:
#Estableciendo parametros (Datos de Phishing de México) y realizando consulta
query_params = {"_where":"(countrycode,eq,MX)"}
response = requests.get("https://phishstats.info:2096/api/phishing?", params=query_params)

#Pasando respuesta a DataFrame
df_phishing_mx = pd.DataFrame.from_records(response.json()) 
df_phishing_mx.head(5)

,id,url,ip,countrycode,countryname,regioncode,regionname,city,zipcode,latitude,longitude,asn,bgp,isp,title,date,date_update,hash,score,host,domain,tld,domain_registered_n_days_ago,screenshot,abuse_contact,ssl_issuer,ssl_subject,alexa_rank_host,alexa_rank_domain,n_times_seen_ip,n_times_seen_host,n_times_seen_domain,http_code,http_server,google_safebrowsing,virus_total,abuse_ch_malware,threat_crowd,threat_crowd_subdomain_count,threat_crowd_votes,vulns,ports,os,tags,technology,page_text
0,5872,http://empleados.morelos.gob.mx/v3/index.php,200.33.79.204,MX,Mexico,,,,,19,-99.01110000,AS22884,200.33.79.0/24,"TOTAL PLAY TELECOMUNICACIONES SA DE CV, MX",None,2016-10-01T03:00:00.000Z,2020-12-08T01:50:24.000Z,81cac894db6e624ef8e0b3e5124181c200e9ab375ec445...,None,empleados.morelos.gob.mx,None,mx,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
1,8996,http://jimaja.com.mx//cb/cibc/security/24/onli...,200.57.64.90,MX,Mexico,,,,,19,-99.01110000,AS13591,200.57.64.0/20,"Mexico Red de Telecomunicaciones, S. de R.L. d...",None,2017-06-20T03:00:00.000Z,2020-12-08T01:50:24.000Z,ed75429dfe713b44865e0db94eef8fca4ce99395cae89d...,None,jimaja.com.mx,None,mx,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
2,9816,http://lacasitadesanangel.iap.org.mx/imass/ado...,148.235.147.69,MX,Mexico,,,,,19,-99.01110000,AS8151,148.235.128.0/19,"Uninet S.A. de C.V., MX",None,2015-10-20T02:00:00.000Z,2020-12-08T01:50:24.000Z,27f50d1869675724e1af97b02586ced9e6a89eed8463c8...,None,lacasitadesanangel.iap.org.mx,None,mx,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
3,9817,http://lacasitadesanangel.iap.org.mx/imass/ado...,148.235.147.69,MX,Mexico,,,,,19,-99.01110000,AS8151,148.235.128.0/19,"Uninet S.A. de C.V., MX",None,2015-10-19T02:00:00.000Z,2020-12-08T01:50:24.000Z,3a66aaa9d6436dd107ebcdfcd4cec64066fc4dd307881b...,None,lacasitadesanangel.iap.org.mx,None,mx,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
4,9818,http://lacasitadesanangel.iap.org.mx/transcrip...,148.235.147.69,MX,Mexico,,,,,19,-99.01110000,AS8151,148.235.128.0/19,"Uninet S.A. de C.V., MX",None,2015-10-20T02:00:00.000Z,2020-12-08T01:50:24.000Z,55a238e2d3c8c0d370469b37b3df6c59773a0ff8fc1949...,None,lacasitadesanangel.iap.org.mx,None,mx,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,


In [ ]:
#Estableciendo parametros (100 datos de Phishing de Canada con PhishScore mayor a 4) y realizando consulta
query_params = {"_where": "(score,gt,4)~and(countrycode,eq,CA)", 
                "_size": 100}
response = requests.get("https://phishstats.info:2096/api/phishing?", params = query_params)

#Pasando respuesta a DataFrame
df_phishing_ca = pd.DataFrame.from_records(response.json()) 
df_phishing_ca.head(5)

,id,url,ip,countrycode,countryname,regioncode,regionname,city,zipcode,latitude,longitude,asn,bgp,isp,title,date,date_update,hash,score,host,domain,tld,domain_registered_n_days_ago,screenshot,abuse_contact,ssl_issuer,ssl_subject,alexa_rank_host,alexa_rank_domain,n_times_seen_ip,n_times_seen_host,n_times_seen_domain,http_code,http_server,google_safebrowsing,virus_total,abuse_ch_malware,threat_crowd,threat_crowd_subdomain_count,threat_crowd_votes,vulns,ports,os,tags,technology,page_text
0,2607846,https://secure.runescape.ca-in.cz//m=weblogin/...,54.39.209.230,CA,Canada,QC,Quebec,Montreal,H3A,46,-73.57480000,AS16276,54.39.0.0/16,"OVH OVH SAS, FR",Website Suspended,2019-01-28T23:02:42.000Z,2020-12-08T01:50:24.000Z,54899e4884b865487988631fea88bb4f3a0e9a089a232d...,5.7,secure.runescape.ca-in.cz,ca-in.cz,cz,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
1,2607847,https://secure.runescape.ca-in.cz/m=weblogin/l...,54.39.209.230,CA,Canada,QC,Quebec,Montreal,H3A,46,-73.57480000,AS16276,54.39.0.0/16,"OVH OVH SAS, FR",Website Suspended,2019-01-28T23:02:43.000Z,2020-12-08T01:50:24.000Z,43101948b30d985719afb7f9b93c5356478f0d9072edfe...,5.7,secure.runescape.ca-in.cz,ca-in.cz,cz,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
2,2607924,https://ej-qhe-z.com/office365/,149.56.133.62,CA,Canada,,,,,44,-79.37160000,AS16276,149.56.0.0/16,"OVH OVH SAS, FR",\nSign in to your account,2019-01-29T00:33:43.000Z,2020-12-08T01:50:24.000Z,cbd47549f1c5491a7ef8c37baea596632a87f933a18a26...,4.3,ej-qhe-z.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
3,2608815,http://dasfdsaf.espeventus.com/sign_in.html?st...,158.69.240.164,CA,Canada,QC,Quebec,Montreal,H3G,46,-73.58330000,AS16276,158.69.0.0/16,"OVH OVH SAS, FR",,2019-01-29T18:33:40.000Z,2020-12-08T01:50:24.000Z,512f1d2500629fd1aff85660313e989f3a4e2e16f95eba...,4.7,dasfdsaf.espeventus.com,None,com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
4,2609073,http://blog.drall.com.br/whatsapp-entenda-as-s...,66.70.190.16,CA,Canada,QC,Quebec,Montreal,H3A,46,-73.57480000,AS16276,66.70.128.0/17,"OVH OVH SAS, FR",WhatsApp: entenda as setas verdes nas mensagen...,2019-01-30T00:32:07.000Z,2020-12-08T01:50:24.000Z,513e84dc9fd6e733789d47001de87b5160abfefad1a788...,4.6,blog.drall.com.br,None,br,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,


In [ ]:
#Estableciendo parametros (Datos de Phishing relacionados a Facebook con PhishScore 
# mayor a 4, que hayan sido registrados antes del 2020, ordenados por fecha) y realizando consulta
query_params = {"_where": "(title,like,~Facebook~)~and(score,gt,4)~and(date,lt,2020-12-31T00:00:00.000Z)",
                "_sort": "-date",
                "_size":100}
response = requests.get("https://phishstats.info:2096/api/phishing?", params = query_params)

#Convirtiendo respuesta a DataFrame
df_facebook = pd.DataFrame.from_records(response.json())
df_facebook.head(5)

,id,url,ip,countrycode,countryname,regioncode,regionname,city,zipcode,latitude,longitude,asn,bgp,isp,title,date,date_update,hash,score,host,domain,tld,domain_registered_n_days_ago,screenshot,abuse_contact,ssl_issuer,ssl_subject,alexa_rank_host,alexa_rank_domain,n_times_seen_ip,n_times_seen_host,n_times_seen_domain,http_code,http_server,google_safebrowsing,virus_total,abuse_ch_malware,threat_crowd,threat_crowd_subdomain_count,threat_crowd_votes,vulns,ports,os,tags,technology,page_text
0,6718890,https://rb.gy/zmctmu?Facebook_Update,76.223.86.4,US,United States,WA,Washington,Seattle,98108,47.5400,-122.3032,AS16509,76.223.86.0/24,"AMAZON-02, US",Facebook Security,2020-12-31T01:38:55.000Z,2020-12-31T02:39:57.000Z,f718601b39ab3f368748dc22d37a601f1296b6b0f5ec1f...,4.4,rb.gy,rb.gy,gy,470.0,None,abuse@rb.gy,None,None,NaN,NaN,49,64,91.0,200,Kestrel,No,Yes,Yes,Yes,2.0,Malicious,None,"80, 443",None,cloud,None,None
1,6718889,https://rb.gy/feu4e9?Facebook_Update,76.223.86.4,US,United States,WA,Washington,Seattle,98108,47.5400,-122.3032,AS16509,76.223.86.0/24,"AMAZON-02, US",Facebook Security,2020-12-31T01:38:52.000Z,2020-12-31T02:39:57.000Z,25ef0fddd9f88bca4863ce65f1da111a73e2a0c9f5f7f1...,4.4,rb.gy,rb.gy,gy,470.0,None,abuse@rb.gy,None,None,NaN,NaN,48,63,90.0,200,Kestrel,No,Yes,Yes,Yes,2.0,Malicious,None,"80, 443",None,cloud,None,None
2,6718888,https://rb.gy/av3txq?Facebook_Update,13.248.219.100,US,United States,WA,Washington,Seattle,98109,47.6348,-122.3451,AS16509,13.248.219.0/24,"AMAZON-02, US",Facebook Security,2020-12-31T01:38:50.000Z,2020-12-31T22:48:05.000Z,5de0c09691fcbd38a061efb8c83810080220a5f5321f0e...,4.4,rb.gy,rb.gy,gy,470.0,None,abuse@rb.gy,None,None,NaN,NaN,40,62,89.0,200,Kestrel,No,Yes,Yes,Yes,2.0,Malicious,None,"80, 443",None,cloud,None,None
3,6718850,https://confirm-fbi-10000089785984687010.tk/Lo...,103.16.198.147,ID,Indonesia,,,,,-6.1750,106.8286,AS13030,103.0.0.0/8,"INIT7 Init7 (Switzerland) Ltd., CH",Facebook Security,2020-12-31T01:37:42.000Z,2020-12-31T02:33:37.000Z,dd8fd4283a42d2de859c404401987a887b0de4da501e21...,5.9,confirm-fbi-10000089785984687010.tk,confirm-fbi-10000089785984687010.tk,tk,NaN,None,abuse@dot.tk abuse@freenom.com,None,None,NaN,NaN,36,0,0.0,200,None,No,Yes,No,No,NaN,None,None,"21, 80, 443, 587, 995, 2082, 2086, 2087",None,starttls,None,None
4,6718849,https://confirm-fbi-10000089785984687009.tk/Lo...,103.16.198.147,ID,Indonesia,,,,,-6.1750,106.8286,AS13030,103.0.0.0/8,"INIT7 Init7 (Switzerland) Ltd., CH",Facebook Security,2020-12-31T01:37:39.000Z,2020-12-31T02:33:18.000Z,4d342e7f799b70114be2f6aee5e83b1781c6cb82ffea1c...,6.4,confirm-fbi-10000089785984687009.tk,confirm-fbi-10000089785984687009.tk,tk,NaN,None,abuse@dot.tk abuse@freenom.com,None,None,NaN,NaN,35,0,0.0,200,None,No,Yes,No,No,NaN,None,None,"21, 80, 443, 587, 995, 2082, 2086, 2087",None,starttls,None,None


In [ ]:
#Importando las funciones necesarias
import requests
import pandas as pd
import datetime
import time

#Definiendo la función
def phishing_data_downloader(n):
  '''
  Funcion que recibe el numero de registros globales (n) que se desean obtener de la 
  pagina de PhishStats y genera/regresa un csv con los datos obtenidos.
  '''

  assert isinstance(n, int), "Valor proporcionado no es un entero"
  assert n > 200 and n < 10000000, "Numero proporcionado no está dentro del rango(mayor a 200 y menor a 10000000)"

  print("Obteniendo datos de PhishStat.info...")
  csv_name = str(datetime.date.today()) + "-phishstats-" + str(n) + ".csv"   #Generando nombre del csv
  

  for i in range(int(n/100)):

    #Definimos los parametros de la solicitud y la URL
    query_params = {"_where": "(score,gt,4)",
                    "_p": str(i),
                    "_sort": "-date",
                    "_size": 100}
    url_data = "https://phishstats.info:2096/api/phishing?"
    
    print(f"Obteniendo pagina {i} de datos...")
    

    #Realizando la solicitud a la primer pagina
    if i == 0:
      response = requests.get(url_data, params = query_params)
      #Confirmamos que la solicitud se haya realizado con exito 
      try:
        response.raise_for_status()
      except requests.exceptions.HTTPError as er:
        return "Error: " + str(er)
      #Guardando el resultado como un DataFrame    
      phishing_df =  pd.DataFrame.from_records(response.json())


    elif i == range(int(n/100))[-1] and n%100 != 0:
      response = requests.get(url_data, params=query_params)
      #Confirmamos que la solicitud se haya realizado con exito 
      try:
        response.raise_for_status()
      except requests.exceptions.HTTPError as er:
        return "Error: " + str(er)
      df_to_combine = pd.DataFrame.from_records(response.json())
      phishing_df = pd.concat([phishing_df, df_to_combine], axis=0) #Concatenando el DataFrame generado
      time.sleep(2)

      #Obteniendo datos faltantes de la ultima pagina
      query_params = {"_where": "(score,gt,4)", #Definimos los parametros para la pagina restante
                      "_p": str(i+1),
                      "_sort": "-date",
                      "_size": n%100}

      print(f"Obteniendo ultima pagina de datos...")
      last_response = requests.get(url_data, params=query_params)

      #Confirmamos que la solicitud se haya realizado con exito 
      try:
        last_response.raise_for_status()
      except requests.exceptions.HTTPError as er:
        return "Error: " + str(er)
      df_to_combine = pd.DataFrame.from_records(last_response.json())
      phishing_df = pd.concat([phishing_df, df_to_combine], axis=0) #Concatenando el DataFrame generado
      time.sleep(2)


    #Realizando la solicitud a las paginas restantes
    else:
      response = requests.get(url_data, query_params)
      #Confirmamos que la solicitud se haya realizado con exito 
      try:
        response.raise_for_status()
      except requests.exceptions.HTTPError as er:
        return "Error: " + str(er)
      df_to_combine = pd.DataFrame.from_records(response.json())
      phishing_df = pd.concat([phishing_df, df_to_combine], axis=0) #Concatenando el DataFrame generado
      time.sleep(2)


  #Transformamos y guardamos el DataFrame obtenido a CSV 
  phishing_df = phishing_df.reset_index(drop=True).to_csv(csv_name)

  print("*" * 20)
  print("Datos obtenidos con éxito")
  print(f"Nombre del archivo: {csv_name}")
  print(f"Fecha y hora de obtención: {datetime.datetime.now()}")
  print("*" * 20)
  return phishing_df

In [ ]:
#Obteniendo los 675 registros de Phishing mas recientes
phishing_data_downloader(675)

Obteniendo datos de PhishStat.info...
Obteniendo pagina 0 de datos...
Obteniendo pagina 1 de datos...
Obteniendo pagina 2 de datos...
Obteniendo pagina 3 de datos...
Obteniendo pagina 4 de datos...
Obteniendo pagina 5 de datos...
Obteniendo ultima pagina de datos...
********************
Datos obtenidos con éxito
Nombre del archivo: 2021-09-13-phishstats-675.csv
Fecha y hora de obtención: 2021-09-13 19:11:53.822155
********************
